In [1]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pickle
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, GlobalAveragePooling2D, ReLU, Dense, DepthwiseConv2D
from tensorflow.keras.models import Model

In [2]:
y_test=np.load('y_testC.npy')
X_test=np.load('X_testC.npy')

In [3]:
y_train=np.load('y_trainC.npy')
X_train=np.load('X_trainC.npy')

In [4]:
X_train = np.array((X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train)))
X_test = np.array((X_test-np.min(X_test))/(np.max(X_test)-np.min(X_test)))
X_train = X_train/np.std(X_train)
X_test = X_test/np.std(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [9]:
np.save("X_trainN", X_train)
np.save("X_testN", X_test)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=123)
print(X_train.shape, X_test.shape, X_val.shape, len(y_train), len(y_test), len(y_val))

(3115, 192, 753, 3) (2761, 192, 753, 3) (1039, 192, 753, 3) 3115 2761 1039


<Font size = 6> <b> MobileNet Model </b></font>

In [13]:
def depth_block(x, strides):
    x = DepthwiseConv2D(3,strides=strides,padding='same',  use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
def single_conv_block(x,filters):
    x = Conv2D(filters, 1,use_bias=False)(x)
    x= BatchNormalization()(x)
    x = ReLU()(x)
    return x
def combo_layer(x,repetition, strides):
    x = depth_block(x,strides)
    filters=32
    x = single_conv_block(x, filters)
    return x
def MobileNet(input_shape=(192, 753, 3),n_classes = 4):
    input = Input (input_shape)
    x = Conv2D(32,3,strides=(2,2),padding = 'same', use_bias=False) (input)
    x =  BatchNormalization()(x)
    x = ReLU()(x)
    x = combo_layer(x,64, strides=(1,1))
    x = combo_layer(x,128,strides=(2,2))
    x = combo_layer(x,128,strides=(1,1))
    x = combo_layer(x,256,strides=(2,2))
    x = combo_layer(x,256,strides=(1,1))
    x = combo_layer(x,512,strides=(2,2))
    for _ in range(5):
        x = combo_layer(x,512,strides=(1,1))
    x = combo_layer(x,1024,strides=(2,2))
    x = combo_layer(x,1024,strides=(1,1))
    x = GlobalAveragePooling2D()(x)
    output = Dense(n_classes,activation='softmax')(x)
    model = Model(input, output)
    return model
n_classes = 4
input_shape = (192, 753, 3)
model = MobileNet(input_shape,n_classes)
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 192, 753, 3)]     0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 96, 377, 32)       864       
_________________________________________________________________
batch_normalization_54 (Batc (None, 96, 377, 32)       128       
_________________________________________________________________
re_lu_54 (ReLU)              (None, 96, 377, 32)       0         
_________________________________________________________________
depthwise_conv2d_26 (Depthwi (None, 96, 377, 32)       288       
_________________________________________________________________
batch_normalization_55 (Batc (None, 96, 377, 32)       128       
_________________________________________________________________
re_lu_55 (ReLU)              (None, 96, 377, 32)      

In [14]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [15]:
md = model.fit(X_train, y_train, epochs=5, validation_data= (X_val, y_val))

Epoch 1/5
98/98 [==============================] - 191s 2s/step - loss: 1.2307 - accuracy: 0.4276 - val_loss: 1.2587 - val_accuracy: 0.4793
Epoch 2/5
98/98 [==============================] - 182s 2s/step - loss: 1.1444 - accuracy: 0.5117 - val_loss: 1.2395 - val_accuracy: 0.4793
Epoch 3/5
98/98 [==============================] - 180s 2s/step - loss: 1.1320 - accuracy: 0.5281 - val_loss: 1.2333 - val_accuracy: 0.2955
Epoch 4/5
98/98 [==============================] - 179s 2s/step - loss: 1.1311 - accuracy: 0.5307 - val_loss: 1.2131 - val_accuracy: 0.4793
Epoch 5/5
98/98 [==============================] - 176s 2s/step - loss: 1.1099 - accuracy: 0.5499 - val_loss: 1.3936 - val_accuracy: 0.4793


In [17]:
yt=model.predict(X_test, verbose=0)
yt

array([[0.7203223 , 0.22836722, 0.01949655, 0.03181389],
       [0.7219606 , 0.21741033, 0.02168512, 0.03894397],
       [0.72390413, 0.21171074, 0.02467756, 0.03970763],
       ...,
       [0.7242286 , 0.22536989, 0.02058792, 0.02981369],
       [0.73000777, 0.22212346, 0.02063318, 0.02723565],
       [0.7908352 , 0.17986155, 0.01405977, 0.01524353]], dtype=float32)

In [18]:
for i in range(yt.shape[0]):
    print(str(y_train[i])+"  "+str(np.argmax(yt[i])))

2  0
0  0
0  0
0  0
1  0
1  0
0  0
1  0
0  0
0  0
0  0
0  0
0  0
0  0
1  0
1  0
0  0
3  0
0  0
1  0
1  0
1  0
0  0
2  0
2  0
2  0
1  0
3  0
0  0
1  0
1  0
0  0
0  0
0  0
1  0
0  0
0  0
1  0
2  0
1  0
1  0
1  0
2  0
1  0
3  0
2  0
2  0
0  0
1  0
2  0
0  0
0  0
3  0
0  0
2  0
3  0
1  0
1  0
0  0
0  0
1  0
1  0
1  0
1  0
1  0
1  0
0  0
1  0
1  0
0  0
0  0
2  0
1  0
0  0
2  0
1  0
3  0
0  0
1  0
1  0
3  0
0  0
2  0
0  0
1  0
0  0
3  0
0  0
2  0
0  0
1  0
0  0
2  0
0  0
0  0
0  0
1  0
2  0
0  0
1  0
1  0
1  0
0  0
3  0
0  0
0  0
1  0
1  0
0  0
3  0
1  0
0  0
0  0
1  0
0  0
3  0
0  0
0  0
0  0
1  0
0  0
0  0
0  0
0  0
0  0
3  0
0  0
0  0
0  0
0  0
0  0
0  0
0  0
0  0
1  0
1  0
2  0
2  0
0  0
1  0
0  0
3  0
1  0
1  0
0  0
0  0
1  0
2  0
2  0
1  0
0  0
1  0
0  0
1  0
1  0
1  0
0  0
0  0
0  0
1  0
2  0
1  0
0  0
0  0
0  0
0  0
1  0
1  0
0  0
1  0
0  0
0  0
3  0
2  0
0  0
1  0
0  0
1  0
0  0
0  0
1  0
0  0
2  0
0  0
1  0
1  0
0  0
1  0
1  0
2  0
0  0
0  0
2  0
0  0
0  0
1  0
0  0
0  0
1  0
0  0


1  0
0  0
2  0
0  0
0  0
0  0
0  0
0  0
0  0
3  0
0  0
0  0
3  0
2  0
2  0
0  0
3  0
0  0
0  0
1  0
1  0
1  0
1  0
0  0
1  0
0  0
0  0
1  0
0  0
0  0
1  0
1  0
1  0
2  0
0  0
0  0
2  0
2  0
0  0
2  0
1  0
1  0
0  0
0  0
0  0
0  0
0  0
2  0
0  0
0  0
2  0
0  0
1  0
1  0
0  0
2  0
0  0
3  0
0  0
1  0
1  0
0  0
0  0
3  0
0  0
1  0
1  0
0  0
3  0
0  0
0  0
2  0
1  0
1  0
0  0
2  0
0  0
0  0
0  0
3  0
0  0
0  0
1  0
2  0
0  0
0  0
1  0
3  0
1  0
0  0
0  0
1  0
3  0
0  0
3  0
2  0
1  0
0  0
0  0
1  0
0  0
1  0
1  0
0  0
0  0
2  0
0  0
1  0
1  0
1  0
1  0
1  0
1  0
0  0
1  0
2  0
0  0
1  0
2  0
2  0
2  0
0  0
3  0
1  0
2  0
0  0
1  0
0  0
0  0
0  0
0  0
2  0
0  0
0  0
3  0
0  0
1  0
0  0
1  0
1  0
2  0
1  0
0  0
0  0
0  0
0  0
1  0
1  0
3  0
0  0
1  0
0  0
1  0
1  0
0  0
3  0
0  0
2  0
1  0
0  0
3  0
1  0
0  0
1  0
1  0
0  0
1  0
1  0
0  0
0  0
0  0
2  0
1  0
2  0
1  0
0  0
0  0
0  0
0  0
1  0
2  0
1  0
0  0
0  0
0  0
0  0
0  0
2  0
0  0
1  0
0  0
0  0
0  0
0  0
0  0
1  0
1  0
0  0
0  0
0  0
